# Transpile circuit with different mappings

In [5]:
# imports
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

from qiskit_algorithms import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

## Define problem with ansatz

In [6]:
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
problem = driver.run()

In [7]:
mappers = {}
mappers["JW"] = JordanWignerMapper()
mappers["Parity"] = ParityMapper()
mappers["Bravi-Kitaev"] = BravyiKitaevMapper()

## Create ansatz and transpile to ibm_torino

In [8]:
service = QiskitRuntimeService()
# We define a specific backend
torino_backend = service.backend("ibm_torino")

seed = 43

In [10]:
for key in mappers.keys():
    print(f"Generating ansatz, vqe, and adapt-vqe for: {key}")
    qubit_op = mappers[key].map(problem.second_q_ops()[0])
    
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mappers[key],
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mappers[key],
        ),
    )

    print(f"\tANSATZ: Count-Ops: {ansatz.decompose().decompose().decompose().count_ops()} size: {ansatz.decompose().decompose().decompose().size()} depth = {ansatz.decompose().decompose().decompose().depth()}")

    # transpile ansatz
    pm = generate_preset_pass_manager(
        backend=torino_backend,
        optimization_level=3,
        seed_transpiler=seed,
        layout_method="sabre",
    )

    circuit_opt = pm.run(ansatz)
    circuit_opt.draw("mpl", fold=False, idle_wires=False)
    print(f"\tANSATZ transpiled: Count-Ops: {circuit_opt.decompose().decompose().decompose().count_ops()} size: {circuit_opt.decompose().decompose().decompose().size()} depth = {circuit_opt.decompose().decompose().decompose().depth()}\n\n")



Generating ansatz, vqe, and adapt-vqe for: JW
	ANSATZ: Count-Ops: OrderedDict([('h', 80), ('cx', 56), ('sdg', 20), ('s', 20), ('rz', 12), ('u', 2)]) size: 190 depth = 97
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 271), ('u', 97), ('cx', 41)]) size: 409 depth = 304


Generating ansatz, vqe, and adapt-vqe for: Parity
	ANSATZ: Count-Ops: OrderedDict([('h', 38), ('cx', 38), ('sdg', 11), ('rz', 11), ('s', 11), ('u', 2), ('ry', 1)]) size: 112 depth = 83
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 331), ('u', 89), ('cx', 44)]) size: 464 depth = 335


Generating ansatz, vqe, and adapt-vqe for: Bravi-Kitaev
	ANSATZ: Count-Ops: OrderedDict([('cx', 38), ('h', 36), ('sdg', 10), ('rz', 10), ('s', 10), ('u', 3), ('ry', 2)]) size: 109 depth = 77
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 273), ('u', 85), ('cx', 39)]) size: 397 depth = 315


